# Konvergencestudy
This notebook provides the reader with a convenient framework for performing convergence studies and testing numerical methods.  
We solve the Navier–Stokes equation for different values of $h$ (the spatial mesh size of $\Omega$) and $T$ (the number of time steps in the space–time mesh).  
By varying $h$ and $T$, both spatial and temporal convergence rates can be examined.


In [1]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import math
import numpy as np
from numpy import exp
from methodsnm.vectorspace import *
from methodsnm.fes import *
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf
#from methodsnm.forms import *

source module for methodsNM imported.


In [2]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]


In [3]:
H = [0.75,0.5]
stabilizationconstants = [0.1,1,10,100]
Ngmesh = []
for maxh in H:
    Ngmesh.append(Mesh(unit_cube.GenerateMesh(maxh=maxh)))
for ngmesh in Ngmesh:
    for Timestep in [2,4]:
        for delta in stabilizationconstants:
            m=2
            T =Timestep
            epsi = 0.1
            mesh = UnstructuredHypertriangleMesh(T,ngmesh)
            from methodsnm.forms import *
            from methodsnm.formint import *
            V1 = P1_Hypertriangle_Space(mesh)
            V2 = P1_Hypertriangle_Space(mesh)
            V3 = P1_Hypertriangle_Space(mesh)
            Q = P1_Hypertriangle_Space(mesh)
            Fes = Productspace([V1,V2,V3,Q]) 
            top = mesh.top_bndry_vertices
            initial = mesh.initial_bndry_vertices
            bndry = list_diff(mesh.bndry_vertices, top)
            freedofs = list_diff(mesh.vertices,bndry)

            nu = 1
            wx, wy, wz = 1, 1, 1
            NU = GlobalFunction(lambda x: nu, mesh = mesh)
            w = ConstantVectorFunction(np.array([wx,wy,wz,1]), mesh = mesh)

            uex1 = lambda x: np.exp(-x[3]) * np.sin(np.pi * x[0]) * np.cos(np.pi * x[1]) * np.cos(np.pi * x[2])
            uex2 = lambda x: -np.exp(-x[3]) * np.cos(np.pi * x[0]) * np.sin(np.pi * x[1]) * np.cos(np.pi * x[2])
            uex3 = lambda x: 0.0
            #pex  = lambda x: np.exp(-x[3]) * np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]) * np.sin(np.pi * x[2])
            pex  = lambda x: 0.0
            f1 = lambda x: np.exp(-x[3]) * (
                (-1.0 + 3.0 * (np.pi**2) * nu) * np.sin(np.pi * x[0]) * np.cos(np.pi * x[1]) * np.cos(np.pi * x[2])
                + np.pi * wx * np.cos(np.pi * x[0]) * np.cos(np.pi * x[1]) * np.cos(np.pi * x[2])
                - np.pi * wy * np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]) * np.cos(np.pi * x[2])
                - np.pi * wz * np.sin(np.pi * x[0]) * np.cos(np.pi * x[1]) * np.sin(np.pi * x[2])
                #- np.pi * np.sin(np.pi * x[1]) * np.sin(np.pi * x[2]) * np.cos(np.pi * x[0])  
            )

            f2 = lambda x : -np.exp(-x[3]) * (
                (1.0 - 3.0 * (np.pi**2) * nu) * np.sin(np.pi * x[1]) * np.cos(np.pi * x[0]) * np.cos(np.pi * x[2])
                + np.pi * wx * np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]) * np.cos(np.pi * x[2])
                - np.pi * wy * np.cos(np.pi * x[0]) * np.cos(np.pi * x[1]) * np.cos(np.pi * x[2])
                + np.pi * wz * np.sin(np.pi * x[1]) * np.sin(np.pi * x[2]) * np.cos(np.pi * x[0])
                #- np.pi * np.sin(np.pi * x[0]) * np.sin(np.pi * x[2]) * np.cos(np.pi * x[1])   
            )

            f3 = lambda x: 0.0

            g = lambda x: 0.0

            fx = GlobalFunction(f1, mesh = mesh)
            fy = GlobalFunction(f2, mesh = mesh)
            fz = GlobalFunction(f3, mesh = mesh)
            g = GlobalFunction(g, mesh = mesh)

            BF = BilinearVectorForm(Fes)
            # Diffusion (für x,y,z)
            for b in range(3):
                BF.add_block_integrator(b, b, LaplaceIntegral_without_time(NU))
                BF.add_block_integrator(b, b, ConvectionIntegral(w))
            for b in range(3):
                BF.add_block_integrator(3, b, DivUQIntegrator())
                BF.add_block_integrator(b, 3, DivVPIntegrator())
            BF.add_block_integrator(3,3,PressureStabilizationIntegral(delta))
            BF.assemble()

            LF = LinearVectorForm(Fes)
            LF.add_block_integrator(0, SourceIntegral(fx))
            LF.add_block_integrator(1, SourceIntegral(fy))
            #LF.add_block_integrator(2, SourceIntegral(fz))
            #LF.add_block_integrator(3, SourceIntegral(g))
            LF.assemble()
            u = FEVectorFunction(Fes)
            u._set({0: (uex1,bndry), 1: (uex2,bndry), 2: (uex3,bndry)})
            res = LF.vector - BF.matrix.dot(u.vector)
            freedofs_global = Fes.get_freedofs({0: bndry, 1: bndry, 2: bndry})
            from methodsnm.solver import solve_on_freedofs
            u.vector += solve_on_freedofs(BF.matrix,res,freedofs_global)
            u1 , u2, u3 , p = u.blocks()

            from methodsnm.forms import compute_difference_L2
            u_ex1 = GlobalFunction(uex1, mesh = mesh)
            u_ex2 = GlobalFunction(uex2, mesh = mesh)
            u_ex3 = GlobalFunction(uex3, mesh = mesh)
            l2diff = compute_difference_L2(u1, u_ex1, mesh, intorder = 5)**2
            l2diff += compute_difference_L2(u2, u_ex2, mesh, intorder = 5)**2
            l2diff += compute_difference_L2(u3, u_ex3, mesh, intorder = 5)**2
            print("L2 difference:for ", Timestep, np.sqrt(l2diff), "stabilization factor:", delta)


L2 difference:for  2 0.3382786812141786 stabilization factor: 0.1
L2 difference:for  2 0.33883565117972897 stabilization factor: 1
L2 difference:for  2 0.3393197811090193 stabilization factor: 10
L2 difference:for  2 0.3394182835590617 stabilization factor: 100
L2 difference:for  4 0.3335845359699078 stabilization factor: 0.1
L2 difference:for  4 0.3337767770130077 stabilization factor: 1
L2 difference:for  4 0.333969988675848 stabilization factor: 10
L2 difference:for  4 0.33404105261790173 stabilization factor: 100
L2 difference:for  2 0.1323226999298461 stabilization factor: 0.1
L2 difference:for  2 0.12201742413034974 stabilization factor: 1
L2 difference:for  2 0.12041619151471014 stabilization factor: 10
L2 difference:for  2 0.1203099986935977 stabilization factor: 100
L2 difference:for  4 0.12357154695986203 stabilization factor: 0.1
L2 difference:for  4 0.108698280033887 stabilization factor: 1
L2 difference:for  4 0.10392772858928892 stabilization factor: 10
L2 difference:for 